<a href="https://colab.research.google.com/github/ShanChartrand/NECDataCleaning/blob/main/NEC_data_cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [278]:
!pip install geopandas
!pip install pypostalcode


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [279]:
!pip install wheel
!pip install numpy
!pip install shapely
!pip install fiona
!pip install pyproj

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Importing file

In [280]:
!pip install pandas
import pandas as pd
df = pd.read_csv('https://raw.githubusercontent.com/ShanChartrand/NECDataCleaning/main/necoriginal.csv')


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Importing libraries

In [281]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import io
import requests

Viewing original number of rows and columns

In [282]:
# Count the number of rows and columns
num_rows = df.shape[0]
num_columns = df.shape[1]

# Print the results
print("Number of rows:", num_rows)
print("Number of columns:", num_columns)

Number of rows: 19928
Number of columns: 31


Removing rows 2-8064, before metrics such as Jitter, Latency, and Packet loss were reliably measured

In [283]:
# Read the CSV file
df = pd.read_csv('https://raw.githubusercontent.com/ShanChartrand/NECDataCleaning/main/necoriginal.csv')

# Remove rows 2 through 8064
df.drop(df.index[1:8064], inplace=True)

# Save the updated DataFrame
df.to_csv('https://raw.githubusercontent.com/ShanChartrand/NECDataCleaning/main/necoriginal.csv', index=False)

Counting the new number of rows & columns after removing rows 2-8064

In [284]:
# Count the number of rows and columns
num_rows = df.shape[0]
num_columns = df.shape[1]

# Print the results
print("Number of rows:", num_rows)
print("Number of columns:", num_columns)

Number of rows: 11865
Number of columns: 31


Looking at the % of missing items in columns

In [285]:
# Calculate the percentage of missing cells in each column
missing_percentages = df.isna().sum() / len(df) * 100

# Print the result
print(missing_percentages)

TEST ID                       0.000000
TEST DATE / UTC HOUR          0.000000
GEO LATITUDE                  0.000000
GEO LONGITUDE                 0.000000
DOWNLOAD SPEED                0.000000
UPLOAD SPEED                  0.000000
JITTER                        0.008428
LATENCY                       0.008428
PACKET LOSS                   0.008428
MOBILE TEST                   0.000000
ISP                           0.000000
CLIENT IP                     0.000000
CLIENT CITY                   0.000000
CLIENT PROVINCE               0.000000
CLIENT POSTAL CODE            0.000000
AS NUMBER                     0.008428
IPV6 SUPPORTED                0.000000
DNSSEC SUPPORTED              0.000000
TEST SERVER                   0.000000
browser_test_url              0.008428
CONTRACTED DOWNLOAD SPEED    75.347661
CONTRACTED UPLOAD SPEED      80.311842
CONTRACTED DATA CAP          87.265065
CONTRACTED MONTHLY COST      79.241466
SERVICE TYPE                 73.881163
USER SATISFACTION        

Removing columns with greater than 68% missing values, and then listing the remaining number of rows & columns

In [286]:
# List of columns to remove
columns_to_remove = [
    'CONTRACTED DOWNLOAD SPEED',
    'CONTRACTED UPLOAD SPEED',
    'CONTRACTED DATA CAP',
    'CONTRACTED MONTHLY COST',
    'SERVICE TYPE',
    'USER SATISFACTION',
    'CONNECTION TYPE',
    'USER COMMENTS',
    'HEATMAP_PROJECT_AREA'
]

# Remove the columns
df.drop(columns=columns_to_remove, inplace=True)

# Save the updated DataFrame
df.to_csv('https://raw.githubusercontent.com/ShanChartrand/NECDataCleaning/main/necoriginal.csv', index=False)

# Count the number of rows and columns
num_rows = df.shape[0]
num_columns = df.shape[1]

# Print the results
print("Number of rows:", num_rows)
print("Number of columns:", num_columns)

Number of rows: 11865
Number of columns: 22


Checking for duplicates in the TEST ID column

In [287]:
# Count the number of duplicated TEST ID values
num_duplicates = df.duplicated(subset='TEST ID', keep=False).sum()

# Print the number of duplicates
print(f"There are {num_duplicates} rows with duplicated TEST ID values.")

There are 0 rows with duplicated TEST ID values.


Looking for duplicate rows

In [288]:
# Find duplicate rows based on all columns
duplicates = df[df.duplicated()]

# Display the duplicate rows (if any)
if len(duplicates) > 0:
    print(f"Found {len(duplicates)} duplicate rows:")
    print(duplicates)
else:
    print("No duplicate rows found.")

No duplicate rows found.


Looking at the % of missing items in columns

In [289]:
# Calculate the percentage of missing cells in each column
missing_percentages = df.isna().sum() / len(df) * 100

# Print the result
print(missing_percentages)

TEST ID                  0.000000
TEST DATE / UTC HOUR     0.000000
GEO LATITUDE             0.000000
GEO LONGITUDE            0.000000
DOWNLOAD SPEED           0.000000
UPLOAD SPEED             0.000000
JITTER                   0.008428
LATENCY                  0.008428
PACKET LOSS              0.008428
MOBILE TEST              0.000000
ISP                      0.000000
CLIENT IP                0.000000
CLIENT CITY              0.000000
CLIENT PROVINCE          0.000000
CLIENT POSTAL CODE       0.000000
AS NUMBER                0.008428
IPV6 SUPPORTED           0.000000
DNSSEC SUPPORTED         0.000000
TEST SERVER              0.000000
browser_test_url         0.008428
HEATMAP_MUNICIPALITY     0.000000
HEATMAP_CENSUS_TRACTS    0.000000
dtype: float64


As per above, we can see that the columns that have missing items each have exactly 0.008428%, which indicates there was a test interruption or failure with CIRA. Will remove those rows then confirm the % of missing items per column.

In [290]:
# List of columns to check for missing values
columns_to_check = ['JITTER', 'LATENCY', 'PACKET LOSS', 'AS NUMBER', 'browser_test_url']

# Remove rows with missing values in specified columns
df.dropna(subset=columns_to_check, inplace=True)

# Save the updated DataFrame
df.to_csv('https://raw.githubusercontent.com/ShanChartrand/NECDataCleaning/main/necoriginal.csv', index=False)

# Calculate the percentage of missing cells in each column
missing_percentages = df.isna().sum() / len(df) * 100

# Print the result
print(missing_percentages)

TEST ID                  0.0
TEST DATE / UTC HOUR     0.0
GEO LATITUDE             0.0
GEO LONGITUDE            0.0
DOWNLOAD SPEED           0.0
UPLOAD SPEED             0.0
JITTER                   0.0
LATENCY                  0.0
PACKET LOSS              0.0
MOBILE TEST              0.0
ISP                      0.0
CLIENT IP                0.0
CLIENT CITY              0.0
CLIENT PROVINCE          0.0
CLIENT POSTAL CODE       0.0
AS NUMBER                0.0
IPV6 SUPPORTED           0.0
DNSSEC SUPPORTED         0.0
TEST SERVER              0.0
browser_test_url         0.0
HEATMAP_MUNICIPALITY     0.0
HEATMAP_CENSUS_TRACTS    0.0
dtype: float64


We had 11865 rows, let's see how many were removed

In [291]:
# Count the number of rows
num_rows = df.shape[0]

# Print the results
print("Number of rows:", num_rows)

Number of rows: 11863


We can see that only 2 rows were removed. Now to determine the # of unique ISPs

In [292]:
unique_isp_count = len(df['ISP'].unique())
print("Number of unique ISPs:", unique_isp_count)

Number of unique ISPs: 84


Now we must standardize the ISPs and then count the # of unique ISPs after standardizatioin



In [293]:
def standardize_isp(isp):
    replacements = {
        "Access Communications  Co-operative Limited": "Access Communications Co-operative Limited",
        "Amanah Tech": "Amanah Tech Inc.",
        "Amazon.com, Inc.": "Amazon Technologies Inc.",
        "Bell MTS": "Bell Canada",
        "Bell Mobility": "Bell Canada",
        "Bell Mobility, Inc.": "Bell Canada",
        "Bell MTS Data Centres G.P": "Bell Canada",
        "CIK Telecom": "CIK Telecom INC",
        "Commstream Communications": "Commstream Communications Inc",
        "Corridor Communications, INC.": "Corridor Communications",
        "Manitoba First Nations Education Resource Centre Inc.": "Manitoba First Nations Education Resource Centre",
        "Manitoba Hydro International LTD": "Manitoba Hydro",
        "Manitoba Hydro Telecom": "Manitoba Hydro",
        "MTS Allstream": "MTS Inc.",
        "MTS Inc": "MTS Inc.",
        "MTS Allstream Inc.": "MTS Inc.",
        "MTS Data Centres": "MTS Inc.",
        "Red River College of Applied Arts": "Red River College",
        "Red River College of Applied Arts, Science and Tec": "Red River College",
        "Rogers Cable": "Rogers Communications",
        "Rogers Communications Canada Inc.": "Rogers Communications",
        "Full Throttle Networks": "Full Throttle Networks Inc.",
        "Postmedia Network": "Postmedia Network Inc",
        "Radiant Communications Canada Ltd.": "Radiant Communications Canada",
        "Shaw Communications Inc.": "Shaw Communications",
        "Shaw Telecom G.P.": "Shaw Communications",
        "SpaceX Services": "SpaceX",
        "SpaceX Starlink": "SpaceX",
        "TekSavvy Solutions, Inc.": "TekSavvy Solutions",
        "TELUS Communications Inc": "Telus Communications",
        "TELUS Communications Inc.": "Telus Communications",
        "TeraGo Networks Inc": "TeraGo Networks",
        "TeraGo Networks Inc.": "TeraGo Networks",
        "Total Server Solutions L.L.C": "Total Server Solutions L.L.C",
        "Voyageur Internet Inc": "Voyageur Internet",
        "Xplornet Communications Inc": "Xplornet Communications",
        "Xplornet Communications Inc.": "Xplornet Communications"
    }

    return replacements.get(isp, isp)

# Example usage:
# Assuming you have a DataFrame named 'df' with a column named 'ISP'
df['ISP'] = df['ISP'].apply(standardize_isp)

unique_isp_count = len(df['ISP'].unique())
print("Number of unique ISPs:", unique_isp_count)

Number of unique ISPs: 73


I'm going to list all the different ISPs

In [294]:
unique_isps = df['ISP'].unique()
for isp in unique_isps:
    print(isp)

Shaw Communications
Bell Canada
WiBand Communications
TekSavvy Solutions
Altima Telecom
Cipherkey Exchange Corp.
Red River College
CIK Telecom INC
Voyageur Internet
Xplornet Communications
MERLIN
The Exchange Global Server Center
Amanah Tech Inc.
Primus Telecommunications Canada Inc.
ViaNetTV Inc
Rogers Communications
City of Winnipeg
Stafford Communications, Inc.
Full Throttle Networks Inc.
MTS Inc.
Total Server Solutions L.L.C.
Questzone.Net, Inc.
Telus Communications
RBC Convention Centre Winnipeg
Cogent Communications
BIGPIPE
University of Manitoba
Corus Entertainment Inc
Postmedia Network Inc
Fastnet Communications
T. Grand Networks Inc.
Netminders Data Solution
Manitoba Hydro
TeraGo Networks
Alberta Health Services
South Winnipeg Technical Centre
Swift High Speed.com
VPN
GloboTech Communications
Corridor Communications
LES.NET
VOI NET INC.
Packet Exchange Limited
Performive LLC
Radiant Communications Canada
SpaceX
Access Communications Co-operative Limited
BH Telecom Corp.
MRNet


Looking at for upload/download speeds of 0 mbps

In [295]:
# Count the number of cells equal to zero in the 'DOWNLOAD SPEED' column
download_speed_zeros = (df['DOWNLOAD SPEED'] == 0).sum()

# Count the number of cells equal to zero in the 'UPLOAD SPEED' column
upload_speed_zeros = (df['UPLOAD SPEED'] == 0).sum()

# Print the counts
print("Number of cells equal to zero in 'DOWNLOAD SPEED' column:", download_speed_zeros)
print("Number of cells equal to zero in 'UPLOAD SPEED' column:", upload_speed_zeros)

Number of cells equal to zero in 'DOWNLOAD SPEED' column: 0
Number of cells equal to zero in 'UPLOAD SPEED' column: 5


I now need to remove the 5 rows that have upload speeds of 0, and then list how many cells = 0 to confirm removal was successful

In [296]:
# Count the number of cells equal to zero in the 'UPLOAD SPEED' column before removal
upload_speed_zeros_before = (df['UPLOAD SPEED'] == 0).sum()

# Remove rows where UPLOAD SPEED is equal to 0
df = df[df['UPLOAD SPEED'] != 0]

# Count the number of cells equal to zero in the 'UPLOAD SPEED' column after removal
upload_speed_zeros_after = (df['UPLOAD SPEED'] == 0).sum()

# Save the updated DataFrame
df.to_csv('https://raw.githubusercontent.com/ShanChartrand/NECDataCleaning/main/necoriginal.csv', index=False)

# Print the count before and after removal
print("Number of cells equal to zero in 'UPLOAD SPEED' column before removal:", upload_speed_zeros_before)
print("Number of cells equal to zero in 'UPLOAD SPEED' column after removal:", upload_speed_zeros_after)

Number of cells equal to zero in 'UPLOAD SPEED' column before removal: 5
Number of cells equal to zero in 'UPLOAD SPEED' column after removal: 0


Standarding the all the sub-categories of Winnipeg to be just "Winnipeg"

In [297]:
df['CLIENT CITY'] = df['CLIENT CITY'].str.replace('.*Winnipeg.*', 'Winnipeg', regex=True)

# Get the unique client cities
unique_cities = df['CLIENT CITY'].unique()

# Print the unique client cities
for city in unique_cities:
    print(city)

Winnipeg
Toronto
Vancouver
La Salle
Oakbank
Marathon
Cambridge
Edmonton
Georgetown
Port Hope
Thorold
Old Toronto
Blackstock
Selkirk
Lindsay
Charlemagne
Steinbach
London
Etobicoke
Brandon
Marchand
Baie-d'Urfe
Saint-Quentin
Stonewall
Barrie
Markdale
Hartland
Kenora
Montreal
Calgary
Dowling
Bradford
Dryden
Regina
Thunder Bay
Sydney
North York
Brampton
Mississauga
Heritage Pointe
Cobourg
Hamilton
Cooks Creek
Moose Jaw
Teulon
Headingley
Hague
Markham
Chestermere
Peterborough
Ottawa


We want to remove any rows where LATENCY >= 30,000 and then list how many rows were removed

In [298]:
# Count the number of rows before removing
count_before = len(df)

# Remove rows where latency >= 30,000
df = df[df['LATENCY'] < 30000]

# Count the number of rows after removing
count_after = len(df)

# Calculate the count of removed rows
count_removed = count_before - count_after

# Print the count of removed rows
print("Number of removed rows:", count_removed)

Number of removed rows: 1


Deciding if mean or median should be used as the measure of central tendency for UPLOAD SPEED. Looking at the variability/skewness of the data as a whole, and then specifically in Winnipeg

In [299]:
# Extract the UPLOAD SPEED column as a Series
upload_speed = df['UPLOAD SPEED']

# Calculate the mean and median
mean = upload_speed.mean()
median = upload_speed.median()

# Calculate other statistical measures
range_val = upload_speed.max() - upload_speed.min()
std_dev = upload_speed.std()
skewness = upload_speed.skew()


# Find the minimum and maximum values
min_upload_speed = upload_speed.min()
max_upload_speed = upload_speed.max()

# Print the results
print("Min:", min_upload_speed)
print("Max:", max_upload_speed)
print("Mean:", mean)
print("Median:", median)
print("Range:", range_val)
print("Standard Deviation:", std_dev)
print("Skewness:", skewness)

Min: 0.01
Max: 932.9
Mean: 21.61333642574007
Median: 7.37
Range: 932.89
Standard Deviation: 52.78700467940826
Skewness: 7.410118526389673


Determining central tendency for DOWNLOAD SPEED

In [300]:
# Extract the DOWNLOAD SPEED column as a Series
download_speed = df['DOWNLOAD SPEED']

# Calculate the mean and median
mean = download_speed.mean()
median = download_speed.median()

# Calculate other statistical measures
range_val = download_speed.max() - download_speed.min()
std_dev = download_speed.std()
skewness = download_speed.skew()

# Find the minimum and maximum values
min_download_speed = download_speed.min()
max_download_speed = download_speed.max()

# Print the results
print("Min:", min_download_speed)
print("Max:", max_download_speed)
print("Mean:", mean)
print("Median:", median)
print("Range:", range_val)
print("Standard Deviation:", std_dev)
print("Skewness:", skewness)

Min: 0.01
Max: 941.74
Mean: 95.05224087037193
Median: 53.55
Range: 941.73
Standard Deviation: 113.19806513571231
Skewness: 2.169324903725701


Central tendency for JITTER

In [301]:

# Extract the JITTER column as a Series
jitter = df['JITTER']

# Calculate the mean and median
mean = jitter.mean()
median = jitter.median()

# Calculate other statistical measures
range_val = jitter.max() - jitter.min()
std_dev = jitter.std()
skewness = jitter.skew()

# # Find the minimum and maximum values
min_jitter = jitter.min()
max_jitter = jitter.max()

# Print the results
print("Min:", min_jitter)
print("Max:", max_jitter)
print("Mean:", mean)
print("Median:", median)
print("Range:", range_val)
print("Standard Deviation:", std_dev)
print("Skewness:", skewness)

Min: 0.01
Max: 4368.64
Mean: 7.009313485704647
Median: 2.02
Range: 4368.63
Standard Deviation: 50.52553204031066
Skewness: 60.720424045293136


Examing variabilty in LATENCY

In [302]:
# Extract the LATENCY column as a Series
latency = df['LATENCY']

# Calculate the mean and median
mean = latency.mean()
median = latency.median()

# Calculate other statistical measures
range_val = latency.max() - latency.min()
std_dev = latency.std()
skewness = latency.skew()

# # Find the minimum and maximum values
min_latency = latency.min()
max_latency = latency.max()

# Print the results
print("Min:", min_latency)
print("Max:", max_latency)
print("Mean:", mean)
print("Median:", median)
print("Range:", range_val)
print("Standard Deviation:", std_dev)
print("Skewness:", skewness)

Min: 0.1
Max: 1542.32
Mean: 48.20264822467741
Median: 40.92
Range: 1542.22
Standard Deviation: 34.38554643618535
Skewness: 15.227965290740217


Checking PACKET LOSS variabilty

In [303]:
# Extract the PACKET LOSS column as a Series
packet_loss = df['PACKET LOSS']

# Calculate the mean and median
mean = packet_loss.mean()
median = packet_loss.median()

# Calculate other statistical measures
range_val = packet_loss.max() - packet_loss.min()
std_dev = packet_loss.std()
skewness = packet_loss.skew()

# # Find the minimum and maximum values
min_packet_loss = packet_loss.min()
max_packet_loss = packet_loss.max()

# Print the results
print("Min:", min_packet_loss)
print("Max:", max_packet_loss)
print("Mean:", mean)
print("Median:", median)
print("Range:", range_val)
print("Standard Deviation:", std_dev)
print("Skewness:", skewness)

Min: 0.0
Max: 47.85
Mean: 2.0024837648646368
Median: 0.05
Range: 47.85
Standard Deviation: 4.387685835377621
Skewness: 3.494496286086901


Now looking at the descriptive statistics for Winnipeg, beginning with UPLOAD SPEED

In [304]:
# Filter the DataFrame to include only rows where CLIENT CITY is Winnipeg
filtered_df = df[df['CLIENT CITY'] == 'Winnipeg']

# Calculate the statistical measures for UPLOAD SPEED
upload_speed_stats = filtered_df['UPLOAD SPEED'].describe()

# Calculate the skewness for UPLOAD SPEED
upload_speed_skewness = filtered_df['UPLOAD SPEED'].skew()

# Extract the desired statistical measures
min_value = upload_speed_stats['min']
max_value = upload_speed_stats['max']
mean_value = upload_speed_stats['mean']
median_value = upload_speed_stats['50%']
range_value = upload_speed_stats['max'] - upload_speed_stats['min']
std_deviation = upload_speed_stats['std']
skewness = upload_speed_skewness

# Print the results
print("Min:", min_value)
print("Max:", max_value)
print("Mean:", mean_value)
print("Median:", median_value)
print("Range:", range_value)
print("Standard Deviation:", std_deviation)
print("Skewness:", skewness)

Min: 0.01
Max: 932.9
Mean: 21.84555623047553
Median: 7.48
Range: 932.89
Standard Deviation: 53.06835376461681
Skewness: 7.37323681649779


Descriptive statistics for DOWNLOAD SPEED for Winnipeg

In [305]:
# Filter the DataFrame to include only rows where CLIENT CITY is Winnipeg
filtered_df = df[df['CLIENT CITY'] == 'Winnipeg']

# Calculate the statistical measures for DOWNLOAD SPEED
download_speed_stats = filtered_df['DOWNLOAD SPEED'].describe()

# Calculate the skewness for DOWNLOAD SPEED
download_speed_skewness = filtered_df['DOWNLOAD SPEED'].skew()

# Extract the desired statistical measures
min_value = download_speed_stats['min']
max_value = download_speed_stats['max']
mean_value = download_speed_stats['mean']
median_value = download_speed_stats['50%']
range_value = download_speed_stats['max'] - download_speed_stats['min']
std_deviation = download_speed_stats['std']
skewness = download_speed_skewness

# Print the results
print("Min:", min_value)
print("Max:", max_value)
print("Mean:", mean_value)
print("Median:", median_value)
print("Range:", range_value)
print("Standard Deviation:", std_deviation)
print("Skewness:", skewness)

Min: 0.01
Max: 941.74
Mean: 96.25317598056232
Median: 54.925
Range: 941.73
Standard Deviation: 113.6981958885387
Skewness: 2.141829013372057


Descriptive statistics for JITTER for Winnipeg

In [306]:
# Filter the DataFrame to include only rows where CLIENT CITY is Winnipeg
filtered_df = df[df['CLIENT CITY'] == 'Winnipeg']

# Calculate the statistical measures for JITTER
jitter_stats = filtered_df['JITTER'].describe()

# Calculate the skewness for JITTER
jitter_skewness = filtered_df['JITTER'].skew()

# Extract the desired statistical measures
min_value = jitter_stats['min']
max_value = jitter_stats['max']
mean_value = jitter_stats['mean']
median_value = jitter_stats['50%']
range_value = jitter_stats['max'] - jitter_stats['min']
std_deviation = jitter_stats['std']
skewness = jitter_skewness

# Print the results
print("Min:", min_value)
print("Max:", max_value)
print("Mean:", mean_value)
print("Median:", median_value)
print("Range:", range_value)
print("Standard Deviation:", std_deviation)
print("Skewness:", skewness)

Min: 0.01
Max: 1594.51
Mean: 6.462135543214162
Median: 2.02
Range: 1594.5
Standard Deviation: 29.75041411027887
Skewness: 30.667239463210397


Descriptive statistics for LATENCY for Winnipeg

In [307]:
# Filter the DataFrame to include only rows where CLIENT CITY is Winnipeg
filtered_df = df[df['CLIENT CITY'] == 'Winnipeg']

# Calculate the statistical measures for LATENCY
latency_stats = filtered_df['LATENCY'].describe()

# Calculate the skewness for LATENCY
latency_skewness = filtered_df['LATENCY'].skew()

# Extract the desired statistical measures
min_value = latency_stats['min']
max_value = latency_stats['max']
mean_value = latency_stats['mean']
median_value = latency_stats['50%']
range_value = latency_stats['max'] - latency_stats['min']
std_deviation = latency_stats['std']
skewness = latency_skewness

# Print the results
print("Min:", min_value)
print("Max:", max_value)
print("Mean:", mean_value)
print("Median:", median_value)
print("Range:", range_value)
print("Standard Deviation:", std_deviation)
print("Skewness:", skewness)

Min: 0.18
Max: 1542.32
Mean: 48.01969194724054
Median: 40.9
Range: 1542.1399999999999
Standard Deviation: 34.028092954873095
Skewness: 15.94389819830861


Descriptive statistics for PACKET LOSS for Winnipeg

In [308]:
# Filter the DataFrame to include only rows where CLIENT CITY is Winnipeg
filtered_df = df[df['CLIENT CITY'] == 'Winnipeg']

# Calculate the statistical measures for PACKET LOSS
packet_loss_stats = filtered_df['PACKET LOSS'].describe()

# Calculate the skewness for PACKET LOSS
packet_loss_skewness = filtered_df['PACKET LOSS'].skew()

# Extract the desired statistical measures
min_value = packet_loss_stats['min']
max_value = packet_loss_stats['max']
mean_value = packet_loss_stats['mean']
median_value = packet_loss_stats['50%']
range_value = packet_loss_stats['max'] - packet_loss_stats['min']
std_deviation = packet_loss_stats['std']
skewness = packet_loss_skewness

# Print the results
print("Min:", min_value)
print("Max:", max_value)
print("Mean:", mean_value)
print("Median:", median_value)
print("Range:", range_value)
print("Standard Deviation:", std_deviation)
print("Skewness:", skewness)

Min: 0.0
Max: 47.85
Mean: 2.0033825060742796
Median: 0.05
Range: 47.85
Standard Deviation: 4.384114988500783
Skewness: 3.5027357789720512


For Winnipeg only, listing the % of rows and postal codes that meet the "quality internet threshold" of DOWNLOAD SPEED >= 50 and UPLOAD SPEED >= 10 and LATENCY <= 50 and JITTER <= 5 and PACKET LOSS <= 0.25

In [309]:
# Filter rows where CLIENT CITY is 'Winnipeg'
winnipeg_df = df[df['CLIENT CITY'] == 'Winnipeg']

# Filter rows based on conditions
filtered_df = winnipeg_df[
    (winnipeg_df['DOWNLOAD SPEED'] >= 50) &
    (winnipeg_df['UPLOAD SPEED'] >= 10) &
    (winnipeg_df['LATENCY'] <= 50) &
    (winnipeg_df['JITTER'] <= 5) &
    (winnipeg_df['PACKET LOSS'] <= 0.25)
]

# Calculate the percentage
percentage = (len(filtered_df) / len(winnipeg_df)) * 100

# Print the percentage
print("Percentage of rows meeting the conditions:", percentage)

# Get the unique CLIENT POSTAL CODEs
postal_codes = filtered_df['CLIENT POSTAL CODE'].unique()

# Print the CLIENT POSTAL CODEs
print("CLIENT POSTAL CODEs meeting the conditions:")
for code in postal_codes:
    print(code)

Percentage of rows meeting the conditions: 10.638667129468935
CLIENT POSTAL CODEs meeting the conditions:
R2W1R7
R3K0N1
R3B2C3
R3R1E8
R3C1N9
R3P0B3
R2G2P7
R3T5W7
R2J1E5
R3Y1T9
R2H0Y1
R3L0X9
R3H0J9
R3J1T6
R3T6A8
R3G2R3
R3C1A7
R0G1N0
R3J3X6
R3T5B9
R3Y0G6
R3B1B9
R3C4G2
R2C0P8
R3C0K8
R3T1B8
R2K0P2
R3B1C3
R2C3V5
R2C3L9
R2V1V1
R2N1K1
R2C4J1
R3N1P4
R2P2T7
R2K2X6
R2H0N5
R2C3X4
R2M1S7
R3T4X3
R3J1V1
R3Y0X4
R2N4C9
R3C4L5
R2K2W6
R2C5K4
R2M2R9
R3P1M1
R3P0M1
R2R0V7
R3Y0L8
R3L2L7
R3P2T6
R3P0X4
R2V3J2
R3G1M4
R3M0G2
R3L1R8
R3V1N1
R2X0W7
R3C1A9
R2M3X2
R3J2G4
R3X1V7
R2K0J5
R3N0Y9
R3N1E6
R2H0R2
R3A0H4
R2K1P2
R2W1E2
R3R1B5
R3C1Z5
R3P0J8
R2H0R3
R3J2L6
R3Y0Y2
R2C4Y6
R2H2X7
R2G0Y3
R3C3X2
R3J2K8
R2L1K1
R2L1K3
R3M0L1
R2V2A2
R3K0C8
R2Y1X6
R3E1Y6
R3J2W7
R3X2G1
R2L1M1
R3C1Y5
R3T2N2
R3C2B7
R2M4A1
R2M5R4
R3B1G6
R2Y0Y1
R2P1S8
R2L0K5
R2J0A1
R3T3B8
R2P1J8
R3J1V4
R3R2H4
R2P1Z6
R2N4M7
R3M0P7
R2M2A4
R3T4T7
R2X2W1
R3L2A8
R3B2V5
R2N3Z1
R3M1E7
R3Y0N3
R2M1W8
R3Y0W5
R3M0G9
R3Y1Z4
R3W0H2
R2Y1Z8
R2N4H8
R3P1L5
R2M4S4
R3R3M4
R2L0Z

I want to see these locations on a map of Winnipeg

In [310]:
import pandas as pd
import folium

# Filter the DataFrame based on your criteria
filtered_df = df[
    (df['CLIENT CITY'] == 'Winnipeg') &
    (df['DOWNLOAD SPEED'] >= 50) &
    (df['UPLOAD SPEED'] >= 10) &
    (df['LATENCY'] <= 50) &
    (df['JITTER'] <= 5) &
    (df['PACKET LOSS'] <= 0.25)
]

# Create a map centered on Winnipeg
map_winnipeg = folium.Map(location=[49.8951, -97.1384], zoom_start=12)

# Iterate over the filtered DataFrame and add markers to the map
for index, row in filtered_df.iterrows():
    lat = row['GEO LATITUDE']
    lon = row['GEO LONGITUDE']
    popup = f"Postal Code: {row['CLIENT POSTAL CODE']}"

    # Add a marker for each location
    folium.Marker([lat, lon], popup=popup).add_to(map_winnipeg)

# Display the map
map_winnipeg


In [311]:
# Get the column names
column_names = df.columns.tolist()

# Print the column names
print("Column names:")
for column in column_names:
    print(column)

Column names:
TEST ID
TEST DATE / UTC HOUR
GEO LATITUDE
GEO LONGITUDE
DOWNLOAD SPEED
UPLOAD SPEED
JITTER
LATENCY
PACKET LOSS
MOBILE TEST
ISP
CLIENT IP
CLIENT CITY
CLIENT PROVINCE
CLIENT POSTAL CODE
AS NUMBER
IPV6 SUPPORTED
DNSSEC SUPPORTED
TEST SERVER
browser_test_url
HEATMAP_MUNICIPALITY
HEATMAP_CENSUS_TRACTS


Validating postal codes in Winnipeg against longitude and latitude data

In [312]:
import re

# Function to validate postal codes and check coordinates
def validate_postal_codes():
    invalid_postal_codes = []
    for index, row in df.iterrows():
        postal_code = row['CLIENT POSTAL CODE']
        city = row['CLIENT CITY']

        if city.lower() == 'winnipeg':
            try:
                # Validate the postal code using a regular expression pattern
                pattern = r'^[A-Za-z]\d[A-Za-z]\s?\d[A-Za-z]\d$'
                is_valid = re.match(pattern, postal_code)

                if not is_valid:
                    invalid_postal_codes.append(postal_code)
                else:
                    # Get the expected coordinates for the postal code
                    expected_latitude, expected_longitude = row['GEO LATITUDE'], row['GEO LONGITUDE']

                    # Check if the coordinates match the expected values
                    if expected_latitude != row['GEO LATITUDE'] or expected_longitude != row['GEO LONGITUDE']:
                        invalid_postal_codes.append(postal_code)
            except Exception as e:
                # Error occurred during validation
                print(f"Error: {str(e)}")

    return invalid_postal_codes

# Check and list the invalid postal codes for Winnipeg
invalid_codes = validate_postal_codes()
if invalid_codes:
    print("Invalid postal codes for Winnipeg:")
    for postal_code in invalid_codes:
        print(postal_code)
else:
    print("All postal codes for Winnipeg are valid.")


Invalid postal codes for Winnipeg:
None
None
None
None
R2X2H


Dropping the invalid Postal Code rows

In [313]:
# Get the indices of rows with invalid postal codes
invalid_indices = df[df['CLIENT POSTAL CODE'].isin(['None', 'R2X2H'])].index

# Drop the rows with invalid postal codes
df.drop(invalid_indices, inplace=True)

# Reset the index of the DataFrame
df.reset_index(drop=True, inplace=True)

# Print the updated DataFrame
print(df)


       TEST ID TEST DATE / UTC HOUR  GEO LATITUDE  GEO LONGITUDE  \
0       815901  2019-10-31 11:01:18     49.830394     -97.098467   
1       816052  2019-11-02 15:17:55     49.913728     -97.048878   
2       816416  2019-11-05 19:33:49     49.862478     -97.166401   
3       816624  2019-11-09 21:10:17     49.940894     -97.058845   
4       817032  2019-11-19 21:19:29     49.853743     -97.289909   
...        ...                  ...           ...            ...   
11847  1662931  2023-04-05 16:15:48     49.951400     -97.113300   
11848  1662951  2023-04-05 16:47:50     49.919200     -97.126300   
11849  1663043  2023-04-05 22:54:16     49.896022     -96.974594   
11850  1663045  2023-04-05 22:55:08     49.896022     -96.974594   
11851  1663076  2023-04-06 02:06:32     49.895063     -97.138646   

       DOWNLOAD SPEED  UPLOAD SPEED  JITTER  LATENCY  PACKET LOSS  \
0               16.52          5.97    2.16    42.99         0.98   
1              211.35         14.84    5.18  

Extracting the Postal Prefix into it's own column

In [314]:
# Add a new column "POSTAL PREFIX" containing the first three characters of the postal code
df['POSTAL PREFIX'] = df['CLIENT POSTAL CODE'].str[:3]

# Get the number and names of columns
column_count = len(df.columns)
column_names = df.columns.tolist()

# Print the number and names of columns
print("Number of Columns:", column_count)
print("Column Names:", column_names)

# Get the unique values in the "POSTAL PREFIX" column
unique_values = df['POSTAL PREFIX'].unique()

# Print the unique values
print("Unique Values in 'POSTAL PREFIX' Column:")
for value in unique_values:
    print(value)

# Save the updated DataFrame
df.to_csv('https://raw.githubusercontent.com/ShanChartrand/NECDataCleaning/main/necoriginal.csv', index=False)

Number of Columns: 23
Column Names: ['TEST ID', 'TEST DATE / UTC HOUR', 'GEO LATITUDE', 'GEO LONGITUDE', 'DOWNLOAD SPEED', 'UPLOAD SPEED', 'JITTER', 'LATENCY', 'PACKET LOSS', 'MOBILE TEST', 'ISP', 'CLIENT IP', 'CLIENT CITY', 'CLIENT PROVINCE', 'CLIENT POSTAL CODE', 'AS NUMBER', 'IPV6 SUPPORTED', 'DNSSEC SUPPORTED', 'TEST SERVER', 'browser_test_url', 'HEATMAP_MUNICIPALITY', 'HEATMAP_CENSUS_TRACTS', 'POSTAL PREFIX']
Unique Values in 'POSTAL PREFIX' Column:
R2M
R3W
R3M
R2G
R3R
R2V
R2W
R2P
R2N
R3K
R2H
R3J
R3B
R3C
R3P
R2C
R2Y
R3Y
R3G
R3N
R3T
R3L
R2J
R3A
R3H
R2K
R2X
R2R
R2L
R0G
R3V
R3X
R3E
R5A
J0S
R0C


Looking at how many rows in Winnipeg and how many not in Winnipeg.

In [315]:
# Number of rows where CLIENT CITY = Winnipeg
num_rows_winnipeg = len(df[df['CLIENT CITY'] == 'Winnipeg'])

# Number of rows where CLIENT CITY != Winnipeg
num_rows_not_winnipeg = len(df[df['CLIENT CITY'] != 'Winnipeg'])

print(f'Number of rows where CLIENT CITY = Winnipeg: {num_rows_winnipeg}')
print(f'Number of rows where CLIENT CITY != Winnipeg: {num_rows_not_winnipeg}')

Number of rows where CLIENT CITY = Winnipeg: 11519
Number of rows where CLIENT CITY != Winnipeg: 333


Normalizing the 5 key metrics: Upload Speed, Download Speed, Jitter, Latency, and Packet Loss

In [316]:
"""from sklearn.preprocessing import MinMaxScaler

# Select the columns to be normalized
columns_to_normalize = ['UPLOAD SPEED', 'DOWNLOAD SPEED', 'JITTER', 'LATENCY', 'PACKET LOSS']

# Create a MinMaxScaler object
scaler = MinMaxScaler()

# Normalize the selected columns
df[columns_to_normalize] = scaler.fit_transform(df[columns_to_normalize])

# Save the updated DataFrame
df.to_csv('https://raw.githubusercontent.com/ShanChartrand/NECDataCleaning/main/necoriginal.csv', index=False)#"""


"from sklearn.preprocessing import MinMaxScaler\n\n# Select the columns to be normalized\ncolumns_to_normalize = ['UPLOAD SPEED', 'DOWNLOAD SPEED', 'JITTER', 'LATENCY', 'PACKET LOSS']\n\n# Create a MinMaxScaler object\nscaler = MinMaxScaler()\n\n# Normalize the selected columns\ndf[columns_to_normalize] = scaler.fit_transform(df[columns_to_normalize])\n\n# Save the updated DataFrame\ndf.to_csv('https://raw.githubusercontent.com/ShanChartrand/NECDataCleaning/main/necoriginal.csv', index=False)#"

In [317]:
from scipy import stats

# Subset the data for Winnipeg
winnipeg_data = df[df['CLIENT CITY'] == 'Winnipeg']

# Group the data based on POSTAL PREFIX
groups = winnipeg_data.groupby('POSTAL PREFIX')

# Perform ANOVA for each metric
metrics = ['JITTER', 'LATENCY', 'PACKET LOSS', 'UPLOAD SPEED', 'DOWNLOAD SPEED']
for metric in metrics:
    metric_groups = [group[1][metric] for group in groups]
    f_value, p_value = stats.f_oneway(*metric_groups)
    print(f"Metric: {metric}")
    print("F-value:", f_value)
    print("P-value:", p_value)
    print()


Metric: JITTER
F-value: 2.342326006663161
P-value: 1.2909643799234209e-05

Metric: LATENCY
F-value: 16.80469763443363
P-value: 8.318977265698484e-99

Metric: PACKET LOSS
F-value: 42.5524450410545
P-value: 3.277158976617032e-272

Metric: UPLOAD SPEED
F-value: 16.135595810948907
P-value: 3.062605883941461e-94

Metric: DOWNLOAD SPEED
F-value: 26.480821437748606
P-value: 9.502605723152312e-165



These results indicate that the POSTAL PREFIX can be a significant factor in explaining the variations observed in these metrics within the Winnipeg dataset.

---



Removing all the data except Winnipeg data

In [318]:
# Count the initial number of rows
initial_row_count = len(df)
print("Initial number of rows:", initial_row_count)

# Filter the DataFrame to retain only rows where client city is 'Winnipeg'
df_winnipeg = df[df['CLIENT CITY'].str.strip().str.lower() == 'winnipeg']

# Count the number of rows after filtering
winnipeg_row_count = len(df_winnipeg)
print("Number of rows for Winnipeg:", winnipeg_row_count)

# Save the updated DataFrame
df.to_csv('https://raw.githubusercontent.com/ShanChartrand/NECDataCleaning/main/necoriginal.csv', index=False)


Initial number of rows: 11852
Number of rows for Winnipeg: 11519


Listing the number and names of unique client postal prefixes

In [319]:
# Get the unique postal prefixes in the filtered DataFrame
unique_prefixes = df_winnipeg['POSTAL PREFIX'].unique()

# Count the number of unique postal prefixes
prefix_count = len(unique_prefixes)

# Print the number and names of unique postal prefixes
print("Number of unique postal prefixes:", prefix_count)
print("Unique postal prefixes:")
for prefix in unique_prefixes:
    print(prefix)


Number of unique postal prefixes: 36
Unique postal prefixes:
R2M
R3W
R3M
R2G
R3R
R2V
R2W
R2P
R2N
R3K
R2H
R3J
R3B
R3C
R3P
R2C
R2Y
R3Y
R3G
R3N
R3T
R3L
R2J
R3A
R3H
R2K
R2X
R2R
R2L
R0G
R3X
R3E
R3V
R5A
J0S
R0C


Confirming that all non-Winnipeg rows have been removed.

In [320]:
# Number of rows where CLIENT CITY = Winnipeg
num_rows_winnipeg = len(df[df['CLIENT CITY'] == 'Winnipeg'])

# Number of rows where CLIENT CITY != Winnipeg
num_rows_not_winnipeg = len(df[df['CLIENT CITY'] != 'Winnipeg'])

print(f'Number of rows where CLIENT CITY = Winnipeg: {num_rows_winnipeg}')
print(f'Number of rows where CLIENT CITY != Winnipeg: {num_rows_not_winnipeg}')

Number of rows where CLIENT CITY = Winnipeg: 11519
Number of rows where CLIENT CITY != Winnipeg: 333


I need to count the occurence of each postal prefix to make sure there are enough in each to run meaningful analyses

In [321]:
import pandas as pd

# Assuming your dataset is loaded into a DataFrame called 'df' with a column named 'POSTAL PREFIX'
postal_code_counts = df['POSTAL PREFIX'].value_counts()

print(postal_code_counts)


R2M    2450
R2N    1224
R3T     887
R2W     622
R3C     607
R3J     539
R3G     530
R2V     456
R2C     452
R3B     423
R3P     324
R3L     309
R3R     284
R2Y     283
R3M     267
R2K     259
R2G     235
R2J     233
R3Y     185
R2P     168
R3N     164
R2H     160
R2R     136
R3X     126
R3K     104
R2X      96
R3V      93
R3W      66
R2L      60
R3E      58
R3H      35
R3A      12
R5A       2
R0G       1
J0S       1
R0C       1
Name: POSTAL PREFIX, dtype: int64


Removing any Postal Prefix that is <30, which also happens to account for rural, non-Winnipeg/Manitoba prefixes: R3A, R5A, R0G, J0A & R0C

In [333]:
import pandas as pd

# Number of rows before deletion
num_rows_before = len(df)

# Filter rows with specific POSTAL PREFIX values
postal_prefix_filter = ['R3A', 'R5A', 'R0G', 'J0S', 'R0C']
df = df[~df['POSTAL PREFIX'].isin(postal_prefix_filter)].copy()

# Reset the index
df.reset_index(drop=True, inplace=True)

# Save the updated DataFrame
df.to_csv('https://raw.githubusercontent.com/ShanChartrand/NECDataCleaning/main/necoriginal.csv', index=False)

# Number of rows after deletion
num_rows_after = len(df)

print("Number of rows before deletion:", num_rows_before)
print("Number of rows after deletion:", num_rows_after)


Number of rows before deletion: 11852
Number of rows after deletion: 11835


Now I want the % of each postal prefix that meet the criteria for high-quality internet in Winnipeg, sorted highest to lowest

In [334]:
# Step 1: Filter the data for tests in Winnipeg that meet the criteria
filtered_data = df[(df['CLIENT CITY'] == 'Winnipeg') &
                   (df['DOWNLOAD SPEED'] >= 50) &
                   (df['UPLOAD SPEED'] >= 10) &
                   (df['LATENCY'] <= 50) &
                   (df['JITTER'] <= 5) &
                   (df['PACKET LOSS'] <= 0.25)]

# Step 2: Calculate the total number of tests that meet the criteria in Winnipeg
total_tests_winnipeg = len(filtered_data)

# Step 3: Calculate the number of tests in postal prefix R2W
tests_r2w = len(filtered_data[filtered_data['POSTAL PREFIX'] == 'R2W'])

# Step 4: Calculate the percentage of tests in R2W out of all tests in Winnipeg
percentage_r2w = (tests_r2w / total_tests_winnipeg) * 100

print(f"The percentage of tests in Winnipeg that meet the criteria and have postal prefix R2W is: {percentage_r2w}%")

# Step 5: Calculate the breakdown by postal prefix for tests in Winnipeg
prefix_counts = filtered_data['POSTAL PREFIX'].value_counts()
prefix_percentages = (prefix_counts / total_tests_winnipeg) * 100

# Print the breakdown percentages sorted in descending order
print("Breakdown by Postal Prefix:")
print(prefix_percentages.sort_values(ascending=False))


The percentage of tests in Winnipeg that meet the criteria and have postal prefix R2W is: 3.7674037674037675%
Breakdown by Postal Prefix:
R2M    19.410319
R3T    11.957412
R2N     8.190008
R3B     6.060606
R2Y     4.504505
R3C     4.013104
R2W     3.767404
R3Y     3.276003
R3R     3.112203
R2V     2.620803
R3G     2.620803
R3M     2.538903
R3L     2.538903
R3P     2.375102
R2K     2.293202
R2C     2.129402
R2J     2.047502
R3K     1.965602
R3J     1.638002
R2P     1.638002
R3N     1.556102
R2G     1.556102
R2H     1.474201
R2X     1.392301
R2R     1.146601
R2L     1.064701
R3V     0.982801
R3X     0.819001
R3W     0.737101
R3E     0.327600
R3H     0.245700
Name: POSTAL PREFIX, dtype: float64


I am listing the median Upload Speed, Download Speed, Jitter, Latency, and Packet Loss for each Postal Prefix in Winnipeg. US & DS are descending and J, L & PL are ascending

In [335]:
# Calculate the median metrics for each postal prefix
median_metrics = winnipeg_data.groupby('POSTAL PREFIX')[['UPLOAD SPEED', 'DOWNLOAD SPEED', 'JITTER', 'LATENCY', 'PACKET LOSS']].median(numeric_only=True)

# Sort the median metrics as specified
sorted_upload_speed = median_metrics['UPLOAD SPEED'].sort_values(ascending=False)
sorted_download_speed = median_metrics['DOWNLOAD SPEED'].sort_values(ascending=False)
sorted_jitter = median_metrics['JITTER'].sort_values(ascending=True)
sorted_latency = median_metrics['LATENCY'].sort_values(ascending=True)
sorted_packet_loss = median_metrics['PACKET LOSS'].sort_values(ascending=True)

# Print the sorted median metrics
print("Median UPLOAD SPEED for each POSTAL PREFIX (descending order):")
print(sorted_upload_speed)
print()

print("Median DOWNLOAD SPEED for each POSTAL PREFIX (descending order):")
print(sorted_download_speed)
print()

print("Median JITTER for each POSTAL PREFIX (ascending order):")
print(sorted_jitter)
print()

print("Median LATENCY for each POSTAL PREFIX (ascending order):")
print(sorted_latency)
print()

print("Median PACKET LOSS for each POSTAL PREFIX (ascending order):")
print(sorted_packet_loss)
print()


Median UPLOAD SPEED for each POSTAL PREFIX (descending order):
POSTAL PREFIX
R0C    359.180
R3H     93.330
R3A     57.720
R5A     47.820
R0G     19.480
R3K     18.740
R2J     18.370
R3W     14.950
R3Y     14.750
R3P     14.620
R2L     13.720
R3R     13.150
R3E     11.450
R3X     11.215
R2X     10.260
R3V     10.200
R2K     10.010
R3T      9.805
R2Y      9.760
R2W      9.720
R3B      8.900
R2P      8.280
R3G      8.070
R2H      7.640
R3M      7.510
R3C      6.990
R2G      6.950
R2M      6.805
R3L      6.010
R2N      5.185
R3N      5.020
R2V      4.750
R2C      4.365
R3J      2.000
J0S      1.150
R2R      0.840
Name: UPLOAD SPEED, dtype: float64

Median DOWNLOAD SPEED for each POSTAL PREFIX (descending order):
POSTAL PREFIX
R0C    277.670
R3K    186.980
R2Y    166.290
R2C    139.035
R3R     99.300
R3A     97.610
R2L     90.200
R3H     89.270
R2R     88.250
R3P     85.450
R5A     82.395
R3Y     80.125
R3X     77.430
R0G     73.010
R3B     69.945
R3E     69.245
R2J     67.390
R3L     64.67

In [325]:
# Step 1: Filter the data for tests in Winnipeg
winnipeg_data = df[df['CLIENT CITY'] == 'Winnipeg']

# Step 2: Calculate the median metrics for each POSTAL PREFIX
median_metrics = winnipeg_data.groupby('POSTAL PREFIX').median()

# Step 3: Check if any of the median metrics meet the criteria
criteria_met = (median_metrics['DOWNLOAD SPEED'] >= 50) & \
               (median_metrics['UPLOAD SPEED'] >= 10) & \
               (median_metrics['LATENCY'] <= 50) & \
               (median_metrics['JITTER'] <= 5) & \
               (median_metrics['PACKET LOSS'] <= 0.25)

# Step 4: List the POSTAL PREFIX that meet the criteria
postal_prefixes_met_criteria = median_metrics.index[criteria_met].tolist()

print("POSTAL PREFIX that meet the criteria:")
print(postal_prefixes_met_criteria)


POSTAL PREFIX that meet the criteria:
['R0G', 'R2J', 'R2L', 'R2X', 'R3A', 'R3E', 'R3K', 'R3R', 'R3Y']


<ipython-input-325-857984317974>:5: FutureWarning: The default value of numeric_only in DataFrameGroupBy.median is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  median_metrics = winnipeg_data.groupby('POSTAL PREFIX').median()
